In [1]:
import itertools
import numpy as np
from Line import Line
from utility import *
from calibrate_utility import calibrate_cramera, undistort
from binary_threshold import combined_threshold
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import cv2
from collections import deque

In [2]:
mtx, dist = calibrate_cramera()

In [3]:
bad_count = 99 # Initialize the bad count in order to start with sliding window search
def process_image(image):
    """Takes as input a color imge. Then:
    1. Undistort the image.
    2. Combined edge detection.
    3. Transfer to bird eye.
    *4. line detection, smooth. sanity check"""
    global bad_count
    h,w,c = image.shape
    undistorted_img = undistort(image, mtx, dist)
    threshold_img = combined_threshold(undistorted_img)
    birdeye_img = warp(threshold_img)
    if bad_count > 5 or rline.detected or lline.detected:
        bad_count = 0
        sliding_window_search(birdeye_img,h,rline,lline) # 2D image
    else:
        previous_fit_window_search(birdeye_img,rline,lline)
    # Check the distance between the two lines
    if (rline.poly_fit_meter[2]-lline.poly_fit_meter[2]) > 5 or (rline.poly_fit_meter[2]-lline.poly_fit_meter[2]) < 4:
        bad_count += 1 # log the times bad frame encountered
        left_length = len(lline.recent_fits)
        right_length = len(rline.recent_fits)
        lline.recent_fits = deque(itertools.islice(lline.recent_fits,0,left_length-1), maxlen=15)
        rline.recent_fits = deque(itertools.islice(rline.recent_fits,0,right_length-1), maxlen=15)
        lline.recent_fits_meter = deque(itertools.islice(lline.recent_fits_meter,0,left_length-1), maxlen=15)
        rline.recent_fits_meter = deque(itertools.islice(rline.recent_fits_meter,0,right_length-1), maxlen=15)
    output = draw_back(birdeye_img, rline, lline)
    #Add text to image
    font = cv2.FONT_HERSHEY_SIMPLEX
    offset = calculate_offset(h,w,rline,lline)
    if offset >= 0:
        cv2.putText(output, 'Vehicle is {:.2f} (m) to the right'.format(offset), (800, 60), font, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    else:
        cv2.putText(output, 'Vehicle is {:.2f} (m) to the left'.format(-offset), (800, 60), font, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    curvature = (lline.curvature_meter+rline.curvature_meter) / 2
    if curvature < 5000:
        cv2.putText(output, 'Radius of Curvature {:.2f}m '.format(curvature), (800, 100), font, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    else:
        cv2.putText(output, 'Road is straight ', (800, 100), font, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
    combo = cv2.addWeighted(image, 1, output, 1, 0.)
    
    return combo

In [4]:
rline = Line()
lline = Line()
white_output = 'project_video_output.mp4'
clip1 = VideoFileClip("../project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 1260/1261 [06:10<00:00,  3.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 9min 22s, sys: 1min 6s, total: 10min 29s
Wall time: 6min 11s


In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))